# Multiple Inheritance

I was reviewing a teammate's code when I stumbled upon a multiple inheritance - he made a class inheriting from 2 classes.
Intuitively I commented that multiple inheritance is bad, but when he (correctly) asked why - I was having a hard time explaining why exactly is it so bad and when can it get complicated.


<figure style="text-align: center;">
    <img src="assets/we_dont_do_that_here.jpg" alt="" width="">
    <figcaption><em>A real footage of me when I saw the multiple inheritance</em>.</figcaption>
</figure>


So in this notebook we are going to talk about multiple inheritance: how to do it wrong and sometimes correct, and also about the C3 algorithm and mixins.

## MRO and Monotonicity

Consider the following example:

In [6]:
class A: pass


class B(A): pass


class C(A): pass


class D(B, C): pass

# Inheritance graph:
#
#      A
#     / \
#    B   C
#     \ /
#      D
#

This is called a "diamond" - the inheritance graph looks like a diamond shape: `A` is the parent, `B` and `C` are 2 children of it and then `D` is a single child with both `B` and `C` as parents.

This may seem problematic, because we are unsure where `D` get its methods from.
Say `A` implements a `foo` method and `B` and `C` are overriding it, each in its own way; will `D` have the `foo` of `B` or `C`?

In [14]:
class A:
    def foo(self):
        print("A method")


class B(A):
    def foo(self):
        print("B method")


class C(A):
    def foo(self):
        print("C method")


class D(B, C): pass


D().foo()

B method


Well, it's `B` - Python has no problem here.
What's determining which method gets called by `D` is something called the **Method Resolution Order**, or the **MRO** in short.
We can even observe it in the class attributes:

In [10]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

When a `D` method gets called, Python first looks for it in the defined methods of `D`; then if it is not found it looks in inherited classes in the following order: `B`, then `C`, then `A`, then the `object` class that is inherited by everything in Python.

In our case, `B` precedes `C` because this is the order of inheritance in `D`'s definition: `class D(B, C): ...`.

Changing the order in `D` signature would change the MRO:

In [16]:
class D(C, B): pass


D().foo()
D.__mro__

C method


(__main__.D, __main__.C, __main__.B, __main__.A, object)

But there are cases where there is a "disagreement" in the method resolution order, and Python literally won't let us inherit:

In [11]:
class X: pass


class Y: pass


class A(X, Y): pass


class B(Y, X): pass


class C(A, B): pass

TypeError: Cannot create a consistent method resolution
order (MRO) for bases X, Y

What is that *"consistency"* that we fail to create?

When constructing the MRO, we want the following statement to hold:

&nbsp;&nbsp;&nbsp;&nbsp;*If `C1` precedes `C2` in the MRO of some class `C`, then `C1` should also precede `C2` in the MRO of any class inheriting `C`.*

This property is called "monotonicity".
Monotonicity of MROs keeps the behavior of class inheritance simple and intuitive, otherwise it could lead to bugs which are hard to detect.

In our case, `Y` precedes `X` in the MRO of `B` (`B-Y-X`).
Yet, `C` inherits from `A`, so you would expect the method resolution would be done in the order `C-A-X-Y-B` - but then `X` precedes `Y` in a class that inherits from `B`, which exactly violates the monotonicity of `B`.
In fact, there is **no possible MRO** for `C` in this case, no matter the order in which `C` inherits `A` and `B`; therefore Python raises an error that we fully understand now - cannot create a consistent (monotonic) MRO for `X` and `Y`.

## The C3 Linearization algorithm